In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
from utils import flatten_lol,remove_multi_space
import os
import pymongo


In [2]:
%pwd

'/home/taindp/VINBRAIN_INTERNSHIP/heuristic_inference/code'

In [3]:
_DATASET = '/home/taindp/VINBRAIN_INTERNSHIP/heuristic_inference/data/dataset.csv'
dataset = pd.read_csv(_DATASET)
dataset = dataset.fillna('none_symptom')
dataset.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
3,Fungal infection,itching,skin_rash,dischromic _patches,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom,none_symptom


In [4]:
list_cols = list(dataset.columns)
list_cols.pop(0)
list_merge_full = []
for idx in range(len(dataset)):
    if idx < len(dataset):
        df = dataset[idx:idx+1]
    else:
        df = dataset[idx:]
        
    list_merge_row = []
    for col in list_cols:
        symp = list(df[col])[0]
        if symp not in list_merge_row and symp != 'none_symptom':
            list_merge_row.append(symp)
    list_merge_full.append(list_merge_row)

In [5]:
dataset['Symptom'] = list_merge_full
dataset_groupby = dataset.groupby('Disease')['Symptom'].apply(list).reset_index(name='Symptom')
dataset_groupby['Symptom'] = [list(set(flatten_lol(item))) for item in list(dataset_groupby['Symptom'])]
dataset_groupby.head()

,Disease,Symptom
0,(vertigo) Paroymsal Positional Vertigo,"[ loss_of_balance, headache, unsteadiness, ..."
1,AIDS,"[ high_fever, extra_marital_contacts, patche..."
2,Acne,"[ blackheads, skin_rash, pus_filled_pimples,..."
3,Alcoholic hepatitis,"[ history_of_alcohol_consumption, yellowish_s..."
4,Allergy,"[ chills, watering_from_eyes, shivering, co..."


In [6]:
list_sympt_norm = []
for list_sympt in list(dataset_groupby['Symptom']):
    sublist_sympt_norm = []
    for sympt in list_sympt:
        if sympt.startswith(' '):
            norm_sympt = sympt[1:]
        else:
            norm_sympt = sympt
        sublist_sympt_norm.append(norm_sympt)
    list_sympt_norm.append(sublist_sympt_norm)

In [7]:
dataset_groupby['Symptom'] = list_sympt_norm

In [8]:
dict_disease_symp = dataset_groupby.to_dict('records')
dict_disease_symp

[{'Disease': '(vertigo) Paroymsal  Positional Vertigo',
  'Symptom': ['loss_of_balance',
   'headache',
   'unsteadiness',
   'vomiting',
   'nausea',
   'spinning_movements']},
 {'Disease': 'AIDS',
  'Symptom': ['high_fever',
   'extra_marital_contacts',
   'patches_in_throat',
   'muscle_wasting']},
 {'Disease': 'Acne',
  'Symptom': ['blackheads', 'skin_rash', 'pus_filled_pimples', 'scurring']},
 {'Disease': 'Alcoholic hepatitis',
  'Symptom': ['history_of_alcohol_consumption',
   'yellowish_skin',
   'abdominal_pain',
   'fluid_overload',
   'swelling_of_stomach',
   'vomiting',
   'distention_of_abdomen']},
 {'Disease': 'Allergy',
  'Symptom': ['chills',
   'watering_from_eyes',
   'shivering',
   'continuous_sneezing']},
 {'Disease': 'Arthritis',
  'Symptom': ['swelling_joints',
   'stiff_neck',
   'movement_stiffness',
   'painful_walking',
   'muscle_weakness']},
 {'Disease': 'Bronchial Asthma',
  'Symptom': ['cough',
   'fatigue',
   'breathlessness',
   'mucoid_sputum',
   'hi

In [10]:
len(dict_disease_symp)

41

In [9]:
# file_out = open('/home/taindp/VINBRAIN_INTERNSHIP/explore_kb/data/disease_symp.json','w')
# for item in dict_disease_symp:
#     item_str = str(item).replace(r"'",r'"')
#     file_out.write(item_str)
#     file_out.write('\n')